# Predictions

In [1]:
# Import the necessary libraries 
import pandas as pd

In [2]:
# Read in the dataframes 
books = pd.read_csv('../data/processed/books.csv')
ratings = pd.read_csv('../data/processed/ratings.csv')
top_10_SVDpp= pd.read_csv('../data/processed/top_10_100_SVDpp.csv', index_col=0)
top_10_KNNBaseline = pd.read_csv('../data/processed/top_10_100_KNNBaseline.csv', index_col=0)

In [3]:
def top_ten_prediction(user_id, top_10, books):
    if user_id not in top_10.columns: 
        print("User id", user_id, "not in the database.")
        
        # cold_start
        cold = pd.DataFrame(books.sort_values('bayesian_avg', ascending=False)[:10])
        cold['predicted_rating'] = cold['bayesian_avg']
        cold = cold.sort_values(['predicted_rating','bayesian_avg'], ascending=[False, False])
        #print(cold)
        return(cold)
    
    else:
        top = pd.DataFrame()
        book_list = []
        print("Top predicted books for user id", user_id)
        #print(top_10[user_id])
        for book_id in top_10[user_id]:
            to_add = books[books.book_id == eval(book_id)[0]]
            #print(to_add)
            to_add['predicted_rating'] = eval(book_id)[1]
            #print(to_add)
            top = pd.concat([top, to_add]) 
            top = top.sort_values(['predicted_rating','bayesian_avg'], ascending=[False, False])
        #print(top)
    return(top)

In [4]:
top_10_SVDpp.head(3)

,1,2,4,8,11,15,18,22,24,25,...,38075,52199,16102,5242,12791,42977,41773,34531,7553,27329
0,"(5580, 4.549948969575827)","(7947, 4.745737373705782)","(464, 5)","(7947, 4.745737373705782)","(131, 4.921464073986611)","(7947, 4.745737373705782)","(7947, 4.418343341863211)","(5580, 4.69097964215209)","(4483, 4.845745803567019)","(7947, 4.745737373705782)",...,"(7947, 4.745737373705782)","(7947, 4.745737373705782)","(7947, 4.745737373705782)","(1198, 5)","(4, 5)","(464, 4.5841739675612025)","(7947, 4.745737373705782)","(7254, 5)","(9076, 4.97763072520487)","(3753, 5)"
1,"(7254, 4.517363801066359)","(5580, 4.71308779958996)","(4653, 4.905912553724049)","(5580, 4.71308779958996)","(83, 4.778837616619155)","(5580, 4.71308779958996)","(1788, 4.398587957736449)","(964, 4.627362615295945)","(7254, 4.765974205180721)","(5580, 4.71308779958996)",...,"(5580, 4.71308779958996)","(5580, 4.71308779958996)","(5580, 4.71308779958996)","(5580, 4.980494834429621)","(757, 5)","(5376, 4.556107968951171)","(5580, 4.71308779958996)","(1788, 5)","(11, 4.937928686085719)","(964, 5)"
2,"(4483, 4.446975937220179)","(6361, 4.679972395752708)","(5580, 4.863205923717746)","(6361, 4.679972395752708)","(160, 4.773976884170009)","(6361, 4.679972395752708)","(4483, 4.360146396176736)","(1788, 4.62286885710697)","(7947, 4.732955354558092)","(6361, 4.679972395752708)",...,"(6361, 4.679972395752708)","(6361, 4.679972395752708)","(6361, 4.679972395752708)","(9076, 4.958016218721828)","(5580, 5)","(7947, 4.50001995836539)","(6361, 4.679972395752708)","(6361, 5)","(862, 4.878456017368669)","(337, 5)"


In [5]:
top_10_KNNBaseline.head(3)

,1,2,4,8,11,15,18,22,24,25,...,38075,52199,16102,5242,12791,42977,41773,34531,7553,27329
0,"(4676, 4.713498742416295)","(862, 4.777977045440078)","(4676, 4.936617383807282)","(862, 4.777977045440078)","(4676, 4.8650418607368495)","(862, 4.777977045440078)","(4676, 4.720429496663909)","(4676, 4.789614094448979)","(4676, 4.894389533601471)","(862, 4.777977045440078)",...,"(862, 4.777977045440078)","(862, 4.777977045440078)","(862, 4.777977045440078)","(4676, 5)","(8946, 5)","(4676, 4.79362850759539)","(862, 4.777977045440078)","(3753, 5)","(4676, 5)","(3753, 5)"
1,"(7947, 4.5268151122590625)","(1788, 4.748665552320921)","(9076, 4.694545663288844)","(1788, 4.748665552320921)","(7254, 4.587570602492985)","(1788, 4.748665552320921)","(8946, 4.546398709236097)","(9076, 4.61953787434699)","(862, 4.776996838915768)","(1788, 4.748665552320921)",...,"(1788, 4.748665552320921)","(1788, 4.748665552320921)","(1788, 4.748665552320921)","(7947, 5)","(3660, 5)","(9076, 4.61171529146489)","(1788, 4.748665552320921)","(8946, 5)","(7947, 4.849871664863939)","(103, 5)"
2,"(8946, 4.515804818820051)","(6361, 4.729073026644194)","(8946, 4.665666827126575)","(6361, 4.729073026644194)","(7947, 4.587527267822143)","(6361, 4.729073026644194)","(862, 4.511844257051887)","(5580, 4.565944264502711)","(7947, 4.63039878514021)","(6361, 4.729073026644194)",...,"(6361, 4.729073026644194)","(6361, 4.729073026644194)","(6361, 4.729073026644194)","(862, 4.9590492643402335)","(4676, 5)","(5580, 4.600505601082409)","(6361, 4.729073026644194)","(7039, 5)","(5580, 4.835607319836763)","(1380, 5)"


In [6]:
print("7563" in top_10_SVDpp.columns)
top_10_SVDpp.columns


False


Index(['1', '2', '4', '8', '11', '15', '18', '22', '24', '25',
       ...
       '38075', '52199', '16102', '5242', '12791', '42977', '41773', '34531',
       '7553', '27329'],
      dtype='object', length=8757)

### Predict for a user in the database

In [7]:
# Top 10 and bottom 10 raters by id
print(ratings.user_id.value_counts(ascending=False)[:10])
print(ratings.user_id.value_counts(ascending=True)[:10])
print(ratings[ratings.user_id == 114].user_id.value_counts(ascending=False))

12874    200
30944    200
52036    199
12381    199
28158    199
45554    197
6630     197
37834    196
15604    196
7563     196
Name: user_id, dtype: int64
34590    19
43675    20
40753    21
32128    21
51725    21
42736    22
25258    22
38280    22
16470    22
37640    22
Name: user_id, dtype: int64
114    67
Name: user_id, dtype: int64


In [8]:
ratings.groupby(['user_id','rating']).count()

book_id
user_id rating         
1       1             3
        2             8
        3            45
        4            39
        5            22
...                 ...
53423   4            24
        5            44
53424   3             9
        4            61
        5            63

[236277 rows x 1 columns]

#### KNNBaseline

In [9]:
#Predict users top_ten books
top_books = top_ten_prediction("1234", top_10_KNNBaseline, books)
top_books.reset_index(inplace=True)
print([x for x in top_books.title])
top_books[['book_id', 'title','predicted_rating', 'bayesian_avg', 'tag_1', 'tag_2', 'tag_3']]

Top predicted books for user id 1234
['1/4 جرام', 'The Divan', 'The Way of Kings, Part 1 (The Stormlight Archive #1.1)', 'A Court of Mist and Fury (A Court of Thorns and Roses, #2)', 'Preach My Gospel: A Guide To Missionary Service', "There's Treasure Everywhere: A Calvin and Hobbes Collection", 'Just Mercy: A Story of Justice and Redemption', 'The Calvin and Hobbes Lazy Sunday Book', 'ESV Study Bible', 'Locke & Key, Vol. 5: Clockworks']


C:\Users\cfran\AppData\Local\Temp/ipykernel_12720/2892154768.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_add['predicted_rating'] = eval(book_id)[1]


,book_id,title,predicted_rating,bayesian_avg,tag_1,tag_2,tag_3
0,4676,1/4 جرام,4.332759,3.898077,fiction,non-fiction,suspense
1,8946,The Divan,4.229222,3.994216,poetry,classics,art
2,9141,"The Way of Kings, Part 1 (The Stormlight Archi...",4.141386,4.082211,fantasy,fiction,magic
3,1308,A Court of Mist and Fury (A Court of Thorns an...,4.129135,4.399902,magic,fiction,fantasy
4,9076,Preach My Gospel: A Guide To Missionary Service,4.108676,4.012837,religion,non-fiction,nonfiction
5,6361,There's Treasure Everywhere: A Calvin and Hobb...,4.076715,4.165119,comics,humor,fiction
6,3491,Just Mercy: A Story of Justice and Redemption,4.074664,4.133579,non-fiction,nonfiction,memoir
7,5580,The Calvin and Hobbes Lazy Sunday Book,4.071283,4.145851,comics,humor,graphic-novels
8,7947,ESV Study Bible,4.070479,4.020578,christian,religion,classics
9,6642,"Locke & Key, Vol. 5: Clockworks",4.051843,4.071510,graphic-novels,comics,horror


#### SVDpp

In [10]:
#Predict users top_ten books
top_books = top_ten_prediction("1234", top_10_SVDpp, books)
top_books.reset_index(inplace=True)
print([x for x in top_books.title])
top_books[['book_id', 'title','predicted_rating', 'bayesian_avg', 'tag_1', 'tag_2', 'tag_3']]

Top predicted books for user id 1234
['The Calvin and Hobbes Lazy Sunday Book', 'A Little Life', "It's a Magical World: A Calvin and Hobbes Collection", 'Just Mercy: A Story of Justice and Redemption', 'The Essential Calvin and Hobbes: A Calvin and Hobbes Treasury', 'The Calvin and Hobbes Tenth Anniversary Book', 'Calvin and Hobbes', 'The House at Pooh Corner (Winnie-the-Pooh, #2)', 'Locke & Key, Vol. 6: Alpha & Omega', "Maus I: A Survivor's Tale: My Father Bleeds History (Maus, #1)"]


C:\Users\cfran\AppData\Local\Temp/ipykernel_12720/2892154768.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_add['predicted_rating'] = eval(book_id)[1]


,book_id,title,predicted_rating,bayesian_avg,tag_1,tag_2,tag_3
0,5580,The Calvin and Hobbes Lazy Sunday Book,4.251294,4.145851,comics,humor,graphic-novels
1,1198,A Little Life,4.211607,4.108787,fiction,contemporary,romance
2,4483,It's a Magical World: A Calvin and Hobbes Coll...,4.177987,4.233393,comics,humor,fiction
3,3491,Just Mercy: A Story of Justice and Redemption,4.166480,4.133579,non-fiction,nonfiction,memoir
4,1010,The Essential Calvin and Hobbes: A Calvin and ...,4.145894,4.409991,comics,humor,graphic-novels
5,1788,The Calvin and Hobbes Tenth Anniversary Book,4.116489,4.333782,comics,humor,graphic-novels
6,780,Calvin and Hobbes,4.081628,4.433867,comics,humor,graphic-novels
7,1545,"The House at Pooh Corner (Winnie-the-Pooh, #2)",4.067377,4.208042,childrens,fiction,classics
8,8663,"Locke & Key, Vol. 6: Alpha & Omega",4.036267,4.051593,graphic-novels,comics,horror
9,490,Maus I: A Survivor's Tale: My Father Bleeds Hi...,4.025789,4.262934,graphic-novels,graphic-novel,comics


### Real Ratings

In [11]:
real_ratings = ratings[ratings.user_id == 1234]
real_ratings.groupby('rating').count()

,user_id,book_id
rating,,
1,12,12
2,21,21
3,50,50
4,42,42
5,3,3


In [12]:
real_ratings = ratings[ratings.user_id == 1234]
real_ratings = pd.merge(real_ratings, books, on='book_id')
real_ratings = real_ratings.sort_values(['rating','bayesian_avg'], ascending=[False, False])
real_ratings[['book_id', 'title','rating', 'bayesian_avg', 'tag_1', 'tag_2', 'tag_3']].head(10)

,book_id,title,rating,bayesian_avg,tag_1,tag_2,tag_3
69,129,One Flew Over the Cuckoo's Nest,5,4.118052,classics,fiction,classic
107,516,The Amazing Adventures of Kavalier & Clay,5,4.019645,fiction,historical-fiction,historical
121,32,Of Mice and Men,5,3.847676,classics,fiction,classic
70,168,The Stand,4,4.223891,horror,fiction,fantasy
110,67,A Thousand Splendid Suns,4,4.194937,fiction,historical-fiction,contemporary
87,545,The Velveteen Rabbit,4,4.181946,childrens,classics,fiction
114,10,Pride and Prejudice,4,4.174904,classics,fiction,romance
111,11,The Kite Runner,4,4.157428,fiction,historical-fiction,contemporary
49,265,A Tree Grows in Brooklyn,4,4.156616,classics,historical-fiction,fiction
91,409,Fried Green Tomatoes at the Whistle Stop Cafe,4,4.134777,fiction,historical-fiction,chick lit


## Predict for a user not in the database

KNNBaseline and SVDpp should be the same, since they are showing a generic recommendation. 

#### KNNBaseline

In [13]:
#Predict users top_ten books
top_books = top_ten_prediction("37640", top_10_SVDpp, books)
top_books.reset_index(inplace=True)
print([x for x in top_books.title])
top_books[['book_id', 'title','predicted_rating', 'bayesian_avg', 'tag_1', 'tag_2', 'tag_3']]

User id 37640 not in the database.
['Harry Potter and the Deathly Hallows (Harry Potter, #7)', 'Harry Potter Boxset (Harry Potter, #1-7)', 'Words of Radiance (The Stormlight Archive, #2)', 'Calvin and Hobbes', 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)', 'The Name of the Wind (The Kingkiller Chronicle, #1)', 'Harry Potter and the Goblet of Fire (Harry Potter, #4)', 'The Essential Calvin and Hobbes: A Calvin and Hobbes Treasury', 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)', 'A Court of Mist and Fury (A Court of Thorns and Roses, #2)']


,book_id,title,predicted_rating,bayesian_avg,tag_1,tag_2,tag_3
0,25,Harry Potter and the Deathly Hallows (Harry Po...,4.502533,4.502533,fantasy,young-adult,fiction
1,422,"Harry Potter Boxset (Harry Potter, #1-7)",4.479617,4.479617,fiction,fantasy,ya
2,862,"Words of Radiance (The Stormlight Archive, #2)",4.460310,4.460310,fantasy,fiction,magic
3,780,Calvin and Hobbes,4.433867,4.433867,comics,humor,graphic-novels
4,27,Harry Potter and the Half-Blood Prince (Harry ...,4.422748,4.422748,fantasy,young-adult,fiction
5,192,The Name of the Wind (The Kingkiller Chronicle...,4.421306,4.421306,fantasy,fiction,magic
6,24,Harry Potter and the Goblet of Fire (Harry Pot...,4.411219,4.411219,fantasy,young-adult,fiction
7,1010,The Essential Calvin and Hobbes: A Calvin and ...,4.409991,4.409991,comics,humor,graphic-novels
8,18,Harry Potter and the Prisoner of Azkaban (Harr...,4.400003,4.400003,fantasy,young-adult,fiction
9,1308,A Court of Mist and Fury (A Court of Thorns an...,4.399902,4.399902,magic,fiction,fantasy


#### SVDpp

In [14]:
#Predict users top_ten books
top_books = top_ten_prediction("37640", top_10_KNNBaseline, books)
top_books.reset_index(inplace=True)
print([x for x in top_books.title])
top_books[['book_id', 'title','predicted_rating', 'bayesian_avg', 'tag_1', 'tag_2', 'tag_3']]

User id 37640 not in the database.
['Harry Potter and the Deathly Hallows (Harry Potter, #7)', 'Harry Potter Boxset (Harry Potter, #1-7)', 'Words of Radiance (The Stormlight Archive, #2)', 'Calvin and Hobbes', 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)', 'The Name of the Wind (The Kingkiller Chronicle, #1)', 'Harry Potter and the Goblet of Fire (Harry Potter, #4)', 'The Essential Calvin and Hobbes: A Calvin and Hobbes Treasury', 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)', 'A Court of Mist and Fury (A Court of Thorns and Roses, #2)']


,book_id,title,predicted_rating,bayesian_avg,tag_1,tag_2,tag_3
0,25,Harry Potter and the Deathly Hallows (Harry Po...,4.502533,4.502533,fantasy,young-adult,fiction
1,422,"Harry Potter Boxset (Harry Potter, #1-7)",4.479617,4.479617,fiction,fantasy,ya
2,862,"Words of Radiance (The Stormlight Archive, #2)",4.460310,4.460310,fantasy,fiction,magic
3,780,Calvin and Hobbes,4.433867,4.433867,comics,humor,graphic-novels
4,27,Harry Potter and the Half-Blood Prince (Harry ...,4.422748,4.422748,fantasy,young-adult,fiction
5,192,The Name of the Wind (The Kingkiller Chronicle...,4.421306,4.421306,fantasy,fiction,magic
6,24,Harry Potter and the Goblet of Fire (Harry Pot...,4.411219,4.411219,fantasy,young-adult,fiction
7,1010,The Essential Calvin and Hobbes: A Calvin and ...,4.409991,4.409991,comics,humor,graphic-novels
8,18,Harry Potter and the Prisoner of Azkaban (Harr...,4.400003,4.400003,fantasy,young-adult,fiction
9,1308,A Court of Mist and Fury (A Court of Thorns an...,4.399902,4.399902,magic,fiction,fantasy


### Real Ratings for that user

In [15]:
real_ratings = ratings[ratings.user_id == 37640]
real_ratings = pd.merge(real_ratings, books, on='book_id')
real_ratings = real_ratings.sort_values(['rating','bayesian_avg'], ascending=[False, False])
real_ratings[['book_id', 'title','rating', 'bayesian_avg', 'tag_1', 'tag_2', 'tag_3']].head(10)

,book_id,title,rating,bayesian_avg,tag_1,tag_2,tag_3
14,109,Les Misérables,5,4.095763,classics,fiction,historical-fiction
4,125,Hamlet,5,3.995079,classics,plays,fiction
8,2033,عزازيل,5,3.969736,fiction,historical-fiction,historical
1,7823,عائد إلى حيفا,5,3.947721,fiction,historical-fiction,classics
0,1887,Cyrano de Bergerac,5,3.944277,classics,plays,fiction
21,8748,رأيت رام الله,5,3.926300,biography,fiction,history
18,461,"The Good Earth (House of Earth, #1)",5,3.913542,fiction,young-adult,NaN
5,353,Othello,5,3.903060,classics,plays,fiction
2,154,Macbeth,5,3.898857,plays,fiction,classic
15,778,The Hunchback of Notre-Dame,5,3.868710,classics,fiction,classic
